## Pięć zasad efektywnej inżynierii promptów
1. Jasne instrukcje poprawiają trafność.
2. Przykłady stabilizują odpowiedź.
3. Zdefiniowany format odpowiedzi = przewidywalny wynik.
4. Dzielenie na kroki = lepsze i pełniejsze rozwiązania.
5. Testy i weryfikacja = bezpieczeństwo i poprawność.

In [1]:
# Import bibliotek
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

# odel bazowy
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

### Zasada 1 — Jasne instrukcje

In [2]:
# Zły prompt — niejasny, bez roli i oczekiwań
bad_prompt = "Napisz funkcję w Pythonie."
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content)

# Dobry prompt — jasno określona rola i oczekiwania
good_prompt = """Jesteś ekspertem programującym w Pythonie.
Napisz funkcję w Pythonie, która przyjmuje listę liczb całkowitych
i zwraca nową listę zawierającą tylko liczby parzyste.
Dodaj test jednostkowy w pytest."""
print("\n=== Dobry prompt ===")
print(llm.invoke(good_prompt).content)

=== Zły prompt ===
Oczywiście! Jaką funkcję chciałbyś, aby stworzyć? Możesz podać mi więcej szczegółów na temat jej celu, a ja pomogę Ci ją napisać.

=== Dobry prompt ===
Oczywiście! Poniżej znajduje się funkcja w Pythonie, która przyjmuje listę liczb całkowitych i zwraca nową listę zawierającą tylko liczby parzyste. Dodałem również test jednostkowy z użyciem biblioteki `pytest`.

### Funkcja do filtrowania liczb parzystych

```python
def filter_even_numbers(numbers):
    """Funkcja zwraca listę tylko z liczbami parzystymi."""
    return [num for num in numbers if num % 2 == 0]
```

### Test jednostkowy z użyciem pytest

Aby przetestować tę funkcję, możesz użyć następującego kodu testowego:

```python
import pytest

def test_filter_even_numbers():
    assert filter_even_numbers([1, 2, 3, 4, 5, 6]) == [2, 4, 6]
    assert filter_even_numbers([0, -2, -3, -4]) == [0, -2, -4]
    assert filter_even_numbers([1, 3, 5]) == []
    assert filter_even_numbers([]) == []
    assert filter_even_num

### Zasada 2 — Używaj przykładów

In [3]:
# Zadanie: wygeneruj tagi na podstawie treści strony firmy

# Zero-shot
zero_shot = """Podaj tagi opisujące firmę na podstawie tekstu strony:
Firma Lego produkuje zabawki dla dzieci."""
print("=== Zero-shot ===")
print(llm.invoke(zero_shot).content)

# One-shot
one_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.
Przykład:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Teraz:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi:"""
print("\n=== One-shot ===")
print(llm.invoke(one_shot).content)

# Few-shot
few_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.

Przykład 1:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Przykład 2:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi: sport, odzież, obuwie

Teraz:
Tekst: Firma Tesla produkuje samochody elektryczne i magazyny energii.
Tagi:"""
print("\n=== Few-shot ===")
print(llm.invoke(few_shot).content)


=== Zero-shot ===
Oto kilka tagów, które mogą opisać firmę Lego na podstawie podanego tekstu:

- Lego
- zabawki
- dzieci
- produkcja zabawek
- kreatywność
- edukacja
- rozrywka
- konstrukcja
- rozwój dziecka
- marka dziecięca

=== One-shot ===
odzież, buty, sport

=== Few-shot ===
samochody, elektryczność, energia


### Zasada 3 — Zdefiniuj format odpowiedzi
walidacja wejścia i wyjścia z wykorzystaniem biblioteki Pydantic

In [4]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import dotenv

dotenv.load_dotenv()

# Definicja schematu wyjścia (Pydantic)
class CityGuide(BaseModel):
    city: str = Field(..., description="Miasto, którego dotyczy przewodnik")
    summary: str = Field(..., description="Krótki opis miasta (2–3 zdania)")
    must_do: List[str] = Field(..., description="Lista 3–5 rzeczy do zrobienia")

# LLM + structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(CityGuide)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem od podróży. Odpowiadaj po polsku, zwięźle."),
    ("user", "Stwórz krótki przewodnik po {city} dla {days}-dniowej wizyty.")
])

chain = prompt | structured_llm

# Walidacja wejścia Pydantic + wywołanie łańcucha
from pydantic import BaseModel, Field

class GuideRequest(BaseModel):
    city: str = Field(min_length=2)
    days: int = Field(ge=1, le=7)

req = GuideRequest(city="Poznań", days=2)

result: CityGuide = chain.invoke(req.model_dump())  # <- dostajesz OBIEKT Pydantic
print(result)                         # CityGuide(city=..., summary=..., must_do=[...])
print(result.model_dump_json(indent=2))  # JSON gotowy do zapisu/transportu

city='Poznań' summary='Poznań to jedno z najstarszych miast w Polsce, znane z bogatej historii, pięknej architektury i tętniącej życiem atmosfery. To idealne miejsce na krótki wypad, oferujące zarówno zabytki, jak i nowoczesne atrakcje.' must_do=['Odwiedź Stary Rynek i zobacz ratusz z koziołkami', 'Spaceruj po Ostrówie Tumskim i zwiedź Katedrę Poznańską', 'Zrelaksuj się w Parku Cytadela', 'Spróbuj lokalnych specjałów w jednej z restauracji na Starym Mieście', 'Zobacz Muzeum Narodowe z bogatą kolekcją sztuki']
{
  "city": "Poznań",
  "summary": "Poznań to jedno z najstarszych miast w Polsce, znane z bogatej historii, pięknej architektury i tętniącej życiem atmosfery. To idealne miejsce na krótki wypad, oferujące zarówno zabytki, jak i nowoczesne atrakcje.",
  "must_do": [
    "Odwiedź Stary Rynek i zobacz ratusz z koziołkami",
    "Spaceruj po Ostrówie Tumskim i zwiedź Katedrę Poznańską",
    "Zrelaksuj się w Parku Cytadela",
    "Spróbuj lokalnych specjałów w jednej z restauracji na St

### Zasada 4 — Dziel złożone zadania na kroki

In [11]:
# Zły prompt — wszystko naraz
bad_prompt = """Przygotuj trzydniowy plan zwiedzania Poznania z budżetem 300 euro,
uwzględniając atrakcje, restauracje, transport i mapy."""
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content[:600], "...")

# Dobry prompt — krok po kroku
good_step1 = "Wypisz najważniejsze atrakcje kulturalne w Poznaniu z godzinami otwarcia."
step1 = llm.invoke(good_step1).content
print("\n=== Dobry prompt — krok 1 ===")
print(step1[:600], "...")

good_step2 = f"Na podstawie tej listy ułóż plan zwiedzania na 3 dni, maks 4 atrakcje dziennie. Atrakcje: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Dobry prompt — krok 2 ===")
print(step2[:600], "...")


=== Zły prompt ===
Oto trzydniowy plan zwiedzania Poznania z budżetem 300 euro. Plan uwzględnia atrakcje turystyczne, restauracje, transport oraz przybliżone ceny.

### Dzień 1: Stare Miasto i Okolice

**Rano:**
- **Śniadanie:** Kawiarnia "Café La Ruina" (ok. 5 euro)
- **Atrakcja:** Zamek Cesarski (bilet wstępu: 5 euro)
- **Transport:** Spacer do Starego Miasta

**Południe:**
- **Atrakcja:** Rynek i Ratusz (za darmo, obserwacja koziołków o 12:00)
- **Lunch:** Restauracja "Bistro Na Żywo" (ok. 10-15 euro)

**Popołudnie:**
- **Atrakcja:** Katedra na Ostrowie Tumskim (za darmo, opcjonalnie bilet na wieżę: 3 euro)
- ...

=== Dobry prompt — krok 1 ===
Oto niektóre z najważniejszych atrakcji kulturalnych w Poznaniu wraz z godzinami otwarcia. Proszę pamiętać, że godziny mogą się zmieniać, więc warto sprawdzić aktualne informacje przed wizytą.

1. **Stary Rynek** - serce Poznania, gdzie znajduje się ratusz z koziołkami.
   - Godziny otwarcia: Całodobowo (rynek jako przestrzeń publiczna).

2. *

In [5]:
# Dobry prompt — krok po kroku
good_step1 = "Wygeneruj plan z 4 punktami artykułu dla tematu asteroida 3I/Atlas jest obiektem obcych."
step1 = llm.invoke(good_step1).content
print("\n=== Dobry prompt — krok 1 ===")
print(step1[:600], "...")

good_step2 = f"Na podstawie planu wygeneruj pierwszy paragraf: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Dobry prompt — krok 2 ===")
print(step2[:600], "...")

good_step3 = (f"Na podstawie planu i pierwszego paragrafu, wygeneruj drugi paragraf: plan: {step1}" f"pierwszy paragraf: {step2}")
step3 = llm.invoke(good_step3).content
print("\n=== Dobry prompt — krok 3 ===")
print(step3[:600], "...")


=== Dobry prompt — krok 1 ===
Oto plan artykułu na temat asteroidy 3I/Atlas jako obiektu obcych:

1. **Wprowadzenie do asteroidy 3I/Atlas**
   - Krótkie przedstawienie asteroidy 3I/Atlas: odkrycie, charakterystyka i trajektoria.
   - Wyjaśnienie, dlaczego asteroida ta wzbudza zainteresowanie naukowców i entuzjastów astronomii.

2. **Dowody na pochodzenie obce**
   - Analiza cech fizycznych i chemicznych asteroidy, które mogą sugerować jej nieziemskie pochodzenie.
   - Przegląd badań i obserwacji, które wskazują na możliwość, że 3I/Atlas mogła być stworzona przez obce cywilizacje.

3. **Reakcje społeczności naukowej i publi ...

=== Dobry prompt — krok 2 ===
Asteroida 3I/Atlas, odkryta w 2019 roku, zyskała status jednego z najbardziej intrygujących obiektów w naszym Układzie Słonecznym. Jej nietypowa trajektoria oraz unikalne cechy fizyczne i chemiczne przyciągnęły uwagę zarówno naukowców, jak i entuzjastów astronomii. W miarę jak badania nad tym obiektem postępują, coraz więcej osób z

In [8]:
print(step2)

Asteroida 3I/Atlas, odkryta w 2019 roku, zyskała status jednego z najbardziej intrygujących obiektów w naszym Układzie Słonecznym. Jej nietypowa trajektoria oraz unikalne cechy fizyczne i chemiczne przyciągnęły uwagę zarówno naukowców, jak i entuzjastów astronomii. W miarę jak badania nad tym obiektem postępują, coraz więcej osób zaczyna zastanawiać się nad jego pochodzeniem, a niektórzy sugerują, że 3I/Atlas może być dowodem na istnienie obcych cywilizacji. W artykule tym przyjrzymy się bliżej tej fascynującej asteroidzie, analizując jej charakterystykę, dowody na potencjalne nieziemskie pochodzenie oraz reakcje społeczności naukowej i publicznej na teorie dotyczące obcych.


### Zasada 5 — Testuj i weryfikuj wyniki

In [13]:
%pip install langchain numpy langchain-community

  Using cached aiohttp-3.12.15-cp312-cp312-macosx_10_13_x86_64.whl.metadata (7.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp312-cp312-macosx_10_13_x86_64.whl.metadata (18 kB)
  Using cached multidict-6.6.4-cp312-cp312-macosx_10_13_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.2-cp312-cp312-macosx_10_13_x86_64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp312-cp312-macosx_10_13_x86_64.whl.metadata (73 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain.evaluation import load_evaluator
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("embedding_distance", embeddings_model="openai")

result = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="Warszawa jest stolicą Polski"
)

print(result)


{'score': 0.055613485077633085}


In [3]:
# Dobry prompt — krok po kroku
good_step1 = "Przygotuj 3 opisy laptopa MacBook Pro 15' 2019."
step1 = llm.invoke(good_step1).content
print("\n=== Dobry prompt — krok 1 ===")
print(step1)

good_step2 = f"Oceń, który z opisów jest najlepszy dla MacBook Pro 15' 2019: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Dobry prompt — krok 2 ===")
print(step2)


=== Dobry prompt — krok 1 ===
Oczywiście, oto trzy opisy MacBooka Pro 15" z 2019 roku:

**Opis 1:**
MacBook Pro 15" z 2019 roku to potężne narzędzie dla profesjonalistów, którzy potrzebują niezawodności i wydajności. Wyposażony w sześciordzeniowy procesor Intel Core i7 dziewiątej generacji, zapewnia niesamowitą moc obliczeniową, idealną do zaawansowanej edycji wideo, projektowania graficznego i programowania. Jego wyświetlacz Retina z technologią True Tone oferuje doskonałe odwzorowanie kolorów, co czyni go idealnym wyborem dla kreatywnych profesjonalistów. Dodatkowo, system audio z wysokiej jakości głośnikami i długotrwała bateria sprawiają, że MacBook Pro 15" to doskonały wybór do pracy i rozrywki.

**Opis 2:**
MacBook Pro 15" (2019) to urządzenie stworzone z myślą o najbardziej wymagających użytkownikach. Zasilany przez procesor Intel Core i9 dziewiątej generacji i wyposażony w kartę graficzną Radeon Pro, ten laptop jest gotowy sprostać najbardziej wymagającym zadaniom. Posiada tak